In [1]:
import os
from configparser import ConfigParser

import ibis

ini_path = "/Users/ying/OpenTeams/git/qadmin/rpa.ini"
conn = None


# Create connection when requested and maintain one copy.
def get_conn():
    global conn
    if not conn:
        conn = get_db()
    return conn


def config():
    try:
        filename = os.path.expanduser(ini_path)
        section = "postgresql"
    except Exception as error:
        raise Exception(
            """{0} .ini file error, check syntax or .env file for correct path ref:
                https://github.com/Quansight/qadmin-pm-apps#database-connection-config
                """.format(
                error
            )
        )

    parser = ConfigParser()
    parser.read(filename)

    config_info = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            config_info[param[0]] = param[1]
    else:
        raise Exception(
            "{0} section not found in {1}, or {1} does not exist".format(
                section, filename
            )
        )

    return config_info


def get_db():
    if not ini_path:
        raise Exception(
            """No .ini file found for database config. Add it to your .env. ref:
            https://github.com/Quansight/qadmin-pm-apps#database-connection-config
            """
        )

    try:
        params = config()
        conn = ibis.postgres.connect(**params)
        return conn
    except Exception as error:
        raise Exception(
            f"""{ error } \nDatabase connection failure.
            Check rpa.ini credentials or too many connections
            """
        )

In [4]:
import datetime as dt

def get_now(tzone):
    return dt.datetime.now(tzone)

def monday_this_week(tzone=dt.timezone.utc):
    """Utility to get the date of Monday of this week

    Parameters
    ----------
    tzone: dt.timezone
        (Optional) Timezone requested, defaults to UTC

    Returns
    -------
    monday_date: dt.datetime
        Datetime object of Monday of this week
    """
    # get today's date
    today = get_now(tzone)
    # get the day of the week
    today_weekday = today.weekday()
    # get the date of monday of this week
    monday_date = today - dt.timedelta(days=today_weekday)
    # cleanup extraneous timing
    monday_date = monday_date.replace(
        hour=0, minute=0, second=0, microsecond=0
    )

    return monday_date

def extract_lookback_period(df, lookback_period, tzone=dt.timezone.utc):
    """
    Extract out a lookback period (including this week) of a dataframe

    Parameters
    ----------
    df: pd.DataFrame
        Dataframe containing a single, presorted datetime index
    lookback_period: int
        Number of weeks to extract from df (including this week)
    tzone: dt.timezone
        (Optional) Timezone requested, defaults to UTC

    Returns
    -------
    lookback_df: pd.DataFrame
        Dataframe containing only the data from the lookback period
    """
    # sort the datetime index to allow for slicing
    df.sort_index(inplace=True)
    # get todays date
    today = get_now(tzone)
    # set the lookback period
    lookback_period = lookback_period - 1  # including this week
    # get monday's date
    monday_date = monday_this_week()
    # get the begin date for the lookback period
    #   add one to allow slicing a df that has duplicate dates
    start_date = monday_date - dt.timedelta(days=lookback_period * 7 + 1)
    # clean out
    start_date = start_date.replace(hour=0, minute=0, second=0, microsecond=0)
    # extract the lookback period
    lookback_df = df.loc[start_date:today]

    return lookback_df

def get_now(tzone):
    return dt.datetime.now(tzone)


def monday_this_week(tzone=dt.timezone.utc):
    """Utility to get the date of Monday of this week"""
    # get today's date
    today = get_now(tzone)
    # get the day of the week
    today_weekday = today.weekday()
    # get the date of monday of this week
    monday_date = today - dt.timedelta(days=today_weekday)

    # cleanup extraneous timing
    monday_date = monday_date.replace(
        hour=0, minute=0, second=0, microsecond=0
    )

    return monday_date


def get_sub_total_of_columns(data):
    data.loc["Sub Total", :] = data.sum(numeric_only=True, axis=0)
    return data

In [43]:
import datetime as dt

import pandas as pd

import ibis 

conn = get_conn()

def get_time_entry_task_df():

    conn = get_conn()
    time_entry = conn.table("time_entry", schema="openteams")
    project_task = conn.table("project_task", schema="openteams")
    week = time_entry.spent_date.truncate("week").name("week")

    columns = [
        time_entry.employee_id,
        project_task.project_id,
        time_entry.time_spent,
        week,
        time_entry.billable,
    ]
    group_by = ["project_id", "employee_id", "week"]

    query = (
        time_entry.join(
            project_task,
            time_entry.project_task_id == project_task.project_task_id,
        )[columns]
    ).materialize()

    hours = [query.time_spent.sum().name("hours")]

    time_entry_task = (
        query.filter(query.billable)
        .group_by(group_by)
        .aggregate(hours)
        .materialize()
    )

    return time_entry_task


def get_time_entry_and_allocation():
    # Consolidates the time_entry and allocation data
    conn = get_conn()
    time_entry_task = get_time_entry_task_df()
    weekly_allocation = conn.table("weekly_allocation", schema="openteams")
    
    df = time_entry_task.execute()
    print(" ---------- time_entry_df ----- ")
    print("columns: {}".format(df.columns))
    print("types: {}".format(df.dtypes))
    print("len: {}".format(len(df)))
    df.head()
    print(" --------------- ")


    # Coalesce the employee_id, project_id, and week columns
    emp_col = ibis.coalesce(weekly_allocation.employee_id, time_entry_task.employee_id).name('employee_id')
    proj_col = ibis.coalesce(weekly_allocation.project_id, time_entry_task.project_id).name('project_id')
    week_col = ibis.coalesce(weekly_allocation.week, time_entry_task.week).name('week')

    allocation = weekly_allocation.hours.name("allocation")

    display_columns = [
        emp_col,
        proj_col,
        week_col,
        allocation,
        time_entry_task.hours
    ]

    base_data = (
        time_entry_task.outer_join(
            weekly_allocation,
            [
                time_entry_task.employee_id == weekly_allocation.employee_id,
                time_entry_task.project_id == weekly_allocation.project_id,
                time_entry_task.week == weekly_allocation.week,            
            ], 
        )[display_columns]
    ).materialize()

    return base_data

def get_time_entry_and_allocation_with_info():
    # Fills in organization, employee, project information for time_entry_and_allocation

    conn = get_conn()
    base_data = get_time_entry_and_allocation()

    df = base_data.execute()
    print(" ---------- time_entry_and_alloc  ----- ")
    print("columns: {}".format(df.columns))
    print("types: {}".format(df.dtypes))    
    print("len: {}".format(len(df)))
    df.head()
    print(" --------------- ")
    
    
    projects = conn.table("project", schema="openteams")
    organization = conn.table("organization", schema="openteams")
    persons = conn.table("person", schema="openteams")
    employee = conn.table("employee_view", schema="openteams")

    organization_name = organization.name.name("organization_name")
    project_name = projects.name.name("project_name")
    project_id = projects.project_id.name("project_id")

    columns = [
        base_data.week,
        organization_name,
        project_name,
        project_id,
        persons.first_name,
        persons.last_name,
        base_data.allocation,
        base_data.hours,
        base_data.employee_id,
        projects.is_active,
        projects.is_billable,
    ]

    group_by = [
        "week",
        "employee_id",
        "organization_name",
        "project_name",
        "project_id",
        "first_name",
        "last_name",
    ]

    query = (
        base_data.join(
            projects, base_data.project_id == projects.project_id
        )
        .join(employee, base_data.employee_id == employee.employee_id)
        .join(persons, employee.entity_id == persons.entity_id)
        .join(organization, projects.client_id == organization.entity_id)[
            columns
        ]
    ).materialize()

    allocation = [query.allocation.sum().name("allocation")]

    time_entry_info = (
        # Remove query.is_billable filter b/c these projects are listed
        query.filter([query.is_active])
        .group_by(group_by)
        .aggregate(allocation)
        .materialize()
    )

    return time_entry_info

### Test this again

In [30]:
time_entry_task = get_time_entry_task_df()
time_entry_data = time_entry_task.execute()
print(len(time_entry_data))
time_entry_data

8782


/Users/ying/opt/anaconda3/envs/qadmin/lib/python3.8/site-packages/ibis/backends/pandas/client.py:320: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  return column.astype(out_dtype.to_pandas(), errors='ignore')


,project_id,employee_id,week,hours
0,1329,3994,2021-06-28,7.0
1,1329,3994,2021-07-05,19.5
2,1329,3994,2021-07-12,19.5
3,1329,3994,2021-07-19,12.0
4,1329,4004,2021-01-04,23.43
...,...,...,...,...
8777,1546,4122,2022-01-03,2.0
8778,1547,4207,2022-01-03,10.0
8779,1547,4207,2022-01-10,5.5
8780,1549,4131,2022-01-10,4.02


In [37]:
emp_col = ibis.coalesce(weekly_allocation.employee_id, time_entry_task.employee_id).name('employee_id')
proj_col = ibis.coalesce(weekly_allocation.project_id, time_entry_task.project_id).name('project_id')
week_col = ibis.coalesce(weekly_allocation.week, time_entry_task.week).name('week')

allocation = weekly_allocation.hours.name("allocation")

display_columns = [
    emp_col,
    proj_col,
    week_col,
    allocation,
    time_entry_task.hours
]

base_data = (
    time_entry_task.outer_join(
        weekly_allocation,
        [
            time_entry_task.employee_id == weekly_allocation.employee_id,
            time_entry_task.project_id == weekly_allocation.project_id,
            time_entry_task.week == weekly_allocation.week,            
        ], 
    )[display_columns]
).materialize()

In [33]:
res = base_data.execute(limit=None)
print(len(res))
res.head()

8837


/Users/ying/opt/anaconda3/envs/qadmin/lib/python3.8/site-packages/ibis/backends/pandas/client.py:320: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  return column.astype(out_dtype.to_pandas(), errors='ignore')


,employee_id,project_id,week,allocation,hours
0,3994,1329,2021-06-28,NaN,7.0
1,3994,1329,2021-07-05,NaN,19.5
2,3994,1329,2021-07-12,NaN,19.5
3,3994,1329,2021-07-19,NaN,12.0
4,4004,1329,2021-01-04,NaN,23.43


In [34]:
projects = conn.table("project", schema="openteams")
organization = conn.table("organization", schema="openteams")
persons = conn.table("person", schema="openteams")
employee = conn.table("employee_view", schema="openteams")

organization_name = organization.name.name("organization_name")
project_name = projects.name.name("project_name")
project_id = projects.project_id.name("project_id")

columns = [
    base_data.week,
    organization_name,
    project_name,
    project_id,
    persons.first_name,
    persons.last_name,
    base_data.allocation,
    base_data.hours,
    base_data.employee_id,
    projects.is_active,
    projects.is_billable,
]

group_by = [
    "week",
    "employee_id",
    "organization_name",
    "project_name",
    "project_id",
    "first_name",
    "last_name",
]

query = (
    base_data.join(
        projects, base_data.project_id == projects.project_id
    )
    .join(employee, base_data.employee_id == employee.employee_id)
    .join(persons, employee.entity_id == persons.entity_id)
    .join(organization, projects.client_id == organization.entity_id)[
        columns
    ]
).materialize()

allocation = [query.allocation.sum().name("allocation")]

project_allocation = (
    # Remove query.is_billable filter b/c these projects are listed
    query.filter([query.is_active])
    .group_by(group_by)
    .aggregate(allocation)
    .materialize()
)

In [35]:
res_data = query.execute()
print(len(res_data))
res_data.head()

8837


/Users/ying/opt/anaconda3/envs/qadmin/lib/python3.8/site-packages/ibis/backends/pandas/client.py:320: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  return column.astype(out_dtype.to_pandas(), errors='ignore')


,week,organization_name,project_name,project_id,first_name,last_name,allocation,hours,employee_id,is_active,is_billable
0,2021-06-28,"OpenTeams, Inc.",OpenTeams - Software Development & Website,1329,Trent,Oliphant,NaN,7.0,3994,True,True
1,2021-07-05,"OpenTeams, Inc.",OpenTeams - Software Development & Website,1329,Trent,Oliphant,NaN,19.5,3994,True,True
2,2021-07-12,"OpenTeams, Inc.",OpenTeams - Software Development & Website,1329,Trent,Oliphant,NaN,19.5,3994,True,True
3,2021-07-19,"OpenTeams, Inc.",OpenTeams - Software Development & Website,1329,Trent,Oliphant,NaN,12.0,3994,True,True
4,2021-01-04,"OpenTeams, Inc.",OpenTeams - Software Development & Website,1329,Igor,Derke,NaN,23.43,4004,True,True


In [44]:
time_entry_info = get_time_entry_and_allocation_with_info()
df = time_entry_info.execute(limit=None)
print(len(df))
df.head()

/Users/ying/opt/anaconda3/envs/qadmin/lib/python3.8/site-packages/ibis/backends/pandas/client.py:320: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  return column.astype(out_dtype.to_pandas(), errors='ignore')


 ---------- time_entry_df ----- 
columns: Index(['project_id', 'employee_id', 'week', 'hours'], dtype='object')
types: project_id              int32
employee_id             int32
week           datetime64[ns]
hours                  object
dtype: object
 --------------- 
 ---------- time_entry_and_alloc  ----- 
columns: Index(['employee_id', 'project_id', 'week', 'allocation', 'hours'], dtype='object')
types: employee_id             int64
project_id              int64
week           datetime64[ns]
allocation             object
hours                  object
dtype: object
 --------------- 
18404


,week,employee_id,organization_name,project_name,project_id,first_name,last_name,allocation
0,2018-11-26,3995,Unfunded OS Development,JupyterLab,1327,Saul,Shanabrook,4.0
1,2018-11-26,4000,Unfunded OS Development,uarray,1343,Pearu,Peterson,4.0
2,2018-11-26,4023,Unfunded OS Development,uarray,1343,Hameer,Abbasi,8.0
3,2018-12-03,3995,Unfunded OS Development,JupyterLab,1327,Saul,Shanabrook,14.0
4,2018-12-03,3995,Unfunded OS Development,uarray,1343,Saul,Shanabrook,7.0
